In [1]:
import numpy as np

import importlib
import os, sys
sys.path.append(os.path.abspath(".."))

In [2]:
# --------------------------
# Module reload utility cell
# --------------------------
import importlib
import models.heston
import models.doubleheston
import plotting.plot_utils
import utils.hedging

# Reload modules to pick up edits
importlib.reload(models.heston)
importlib.reload(models.doubleheston)
importlib.reload(plotting.plot_utils)
importlib.reload(utils.hedging)

<module 'utils.hedging' from '/home/machine/Desktop/ErdosFinance2025/utils/hedging.py'>

In [3]:
# This tests the non-vectorized version of price_greeks in Heston class
# Extra comparison with delta in Rouah's book (price call= 7.8057, delta = 0.5501)
Heston_RouahDelta=models.heston.Heston(r=0.05, q=0.03, 
                                 kappa=0, 
                                 theta=0.07, 
                                 sigma=0.0001, 
                                 rho=-0.8)
Heston_RouahDelta.price_greeks(Lphi=0.00001, Uphi=50, dphi=0.001, K=100, tau=0.5, S=100, v=0.07)

{'call_price': np.float64(7.805653238583091),
 'delta': np.float64(0.55013819641897)}

In [4]:
import numpy as np
import models.heston

# Value of reference
# Comparison with delta in Rouah's book (price call= 7.8057, delta = 0.5501)

# Create Heston instance
Heston_q_test = models.heston.Heston(
    r=0.05, q=0.03,
    kappa=0, theta=0.07,
    sigma=0.0001, rho=-0.8
)

# Single-path, single-step arrays
S = np.array([[100.0]])
V = np.array([[0.07]])
Tau = np.array([[0.5]])
K = 100.0

# Laguerre nodes to test
nodes_list = [16, 32, 64, 128]

print("Laguerre convergence benchmark for Heston price and delta:\n")
benchmark_results = {}
for nodes in nodes_list:
    quad_params = {'nodes': nodes}
    Greeks = Heston_q_test.price_greeks_vect(
        K=K, Tau=Tau, S=S, V=V,
        quad_rule='laguerre', quad_params=quad_params
    )
    price = Greeks['Price_call'][0,0]
    delta = Greeks['Delta'][0,0]
    benchmark_results[nodes] = (price, delta)
    print(f"Nodes={nodes:3d}: Price={price:.10f}, Delta={delta:.10f}")


Laguerre convergence benchmark for Heston price and delta:

Nodes= 16: Price=7.8056641900, Delta=0.5501382826
Nodes= 32: Price=7.8056641901, Delta=0.5501382826
Nodes= 64: Price=7.8056641901, Delta=0.5501382826
Nodes=128: Price=7.8056641901, Delta=0.5501382826


In [5]:
import numpy as np
import models.heston

# Compares with cf_price and the value in Rouah's book for delta (0.6730), price call = 19.4538
DoubleHeston_test = models.doubleheston.DoubleHeston(
    r=0.03, q=0.0,
    kappa1=0.9, theta1=0.1, sigma1=0.1,
    rho1=-0.5,
    kappa2=1.2, theta2=0.15, sigma2=0.2,
    rho2=-0.5
)

# Single-path, single-step arrays
S = np.array([[61.9]])
V1 = np.array([[0.6**2]])
V2 = np.array([[0.7**2]])
Tau = np.array([[1.0]])
K = 61.9

# Laguerre nodes to test
nodes_list = [4,8,16, 32, 64, 128]

print("Laguerre convergence benchmark for Double Heston price and delta:\n")
benchmark_results = {}
for nodes in nodes_list:
    quad_params = {'nodes': nodes}
    Greeks = DoubleHeston_test.price_greeks_vect(
        K=K, Tau=Tau, S=S, V1=V1, V2=V2,
        quad_rule='laguerre', quad_params=quad_params
    )
    price = Greeks['Price_call'][0,0]
    delta = Greeks['Delta'][0,0]
    benchmark_results[nodes] = (price, delta)
    print(f"Nodes={nodes:3d}: Price={price:.10f}, Delta={delta:.10f}")


Laguerre convergence benchmark for Double Heston price and delta:

Nodes=  4: Price=19.6352964149, Delta=0.6751361046
Nodes=  8: Price=19.4550373272, Delta=0.6730413740
Nodes= 16: Price=19.4537743179, Delta=0.6729670267
Nodes= 32: Price=19.4537815168, Delta=0.6729675139
Nodes= 64: Price=19.4537815067, Delta=0.6729675138
Nodes=128: Price=19.4537815067, Delta=0.6729675138


In [6]:
import numpy as np
import models.heston

# Comparison with delta in Rouah's book (price call= 7.8057, delta = 0.5501)

Heston_q_test = models.heston.Heston(
    r=0.05, q=0.03,
    kappa=0.0, theta=0.07,
    sigma=0.0001, rho=-0.8
)

# Single-path, single-step arrays
S = np.array([[100.0]])
V = np.array([[0.07]])
Tau = np.array([[0.5]])
K = 100.0

# Trapezoidal step sizes to test
dphi_list = [0.2, 0.1, 0.05, 0.01, 0.005, 0.001]

# Laguerre nodes to test
laguerre_nodes_list = [8, 16, 32, 64]

print("Trapezoidal vs Laguerre convergence (Price_call, Delta):\n")
print(f"{'Trapezoid dx':>12} | {'Price':>12} | {'Delta':>12} || {'Laguerre nodes':>14} | {'Price':>12} | {'Delta':>12}")
print("-"*80)

for i in range(max(len(dphi_list), len(laguerre_nodes_list))):
    # Trapezoidal
    if i < len(dphi_list):
        dx = dphi_list[i]
        quad_params = {'Lphi': 1e-20, 'Uphi': 100, 'dphi': dx}
        Greeks = Heston_q_test.price_greeks_vect(K=K, Tau=Tau, S=S, V=V,
                                                quad_rule='trapezoidal', quad_params=quad_params)
        price_trap = Greeks['Price_call'][0,0]
        delta_trap = Greeks['Delta'][0,0]
    else:
        dx = price_trap = delta_trap = ""

    # Laguerre
    if i < len(laguerre_nodes_list):
        nodes = laguerre_nodes_list[i]
        quad_params = {'nodes': nodes}
        Greeks = Heston_q_test.price_greeks_vect(K=K, Tau=Tau, S=S, V=V,
                                                quad_rule='laguerre', quad_params=quad_params)
        price_lag = Greeks['Price_call'][0,0]
        delta_lag = Greeks['Delta'][0,0]
    else:
        nodes = price_lag = delta_lag = ""

    print(f"{str(dx):>12} | {str(price_trap):>12} | {str(delta_trap):>12} || {str(nodes):>14} | {str(price_lag):>12} | {str(delta_lag):>12}")


Trapezoidal vs Laguerre convergence (Price_call, Delta):

Trapezoid dx |        Price |        Delta || Laguerre nodes |        Price |        Delta
--------------------------------------------------------------------------------
         0.2 | 7.805796186800514 | 0.5501382826498594 ||              8 | 7.805662144264232 | 0.5501382660536968
         0.1 | 7.805730188427404 | 0.5501382826498654 ||             16 | 7.805664190039153 | 0.5501382826497487
        0.05 | 7.805697189240448 | 0.5501382826498635 ||             32 | 7.8056641900528945 | 0.5501382826498605
        0.01 | 7.805670789890648 | 0.5501382826498616 ||             64 | 7.80566419005342 | 0.5501382826498676
       0.005 | 7.805667489972045 | 0.550138282649862 ||                |              |             
       0.001 | 7.8056648500370756 | 0.5501382826498618 ||                |              |             


In [7]:
import numpy as np
import models.heston

# Compares with cf_price and the value in Rouah's book for delta (0.6730), price call = 19.4538
DoubleHeston_test = models.doubleheston.DoubleHeston(
    r=0.03, q=0.0,
    kappa1=0.9, theta1=0.1, sigma1=0.1,
    rho1=-0.5,
    kappa2=1.2, theta2=0.15, sigma2=0.2,
    rho2=-0.5
)

# Single-path, single-step arrays
S = np.array([[61.9]])
V1 = np.array([[0.6**2]])
V2 = np.array([[0.7**2]])
Tau = np.array([[1.0]])
K = 61.9

# Trapezoidal step sizes to test
dphi_list = [0.2,0.1, 0.05, 0.01, 0.005, 0.001, 0.0005, 0.0001]

# Laguerre nodes to test
laguerre_nodes_list = [8, 16, 32, 64]

print("Double Heston: Trapezoidal vs Laguerre convergence (Price_call, Delta):\n")
print(f"{'Trapezoid dx':>12} | {'Price':>12} | {'Delta':>12} || {'Laguerre nodes':>14} | {'Price':>12} | {'Delta':>12}")
print("-"*80)

for i in range(max(len(dphi_list), len(laguerre_nodes_list))):
    # Trapezoidal
    if i < len(dphi_list):
        dx = dphi_list[i]
        quad_params = {'Lphi': 1e-5, 'Uphi': 100, 'dphi': dx}
        Greeks = DoubleHeston_test.price_greeks_vect(K=K, Tau=Tau, S=S, V1=V1, V2=V2,
                                                     quad_rule='trapezoidal', quad_params=quad_params)
        price_trap = Greeks['Price_call'][0,0]
        delta_trap = Greeks['Delta'][0,0]
    else:
        dx = price_trap = delta_trap = ""

    # Laguerre
    if i < len(laguerre_nodes_list):
        nodes = laguerre_nodes_list[i]
        quad_params = {'nodes': nodes}
        Greeks = DoubleHeston_test.price_greeks_vect(K=K, Tau=Tau, S=S, V1=V1, V2=V2,
                                                     quad_rule='laguerre', quad_params=quad_params)
        price_lag = Greeks['Price_call'][0,0]
        delta_lag = Greeks['Delta'][0,0]
    else:
        nodes = price_lag = delta_lag = ""

    print(f"{str(dx):>12} | {str(price_trap):>12} | {str(delta_trap):>12} || {str(nodes):>14} | {str(price_lag):>12} | {str(delta_lag):>12}")


Double Heston: Trapezoidal vs Laguerre convergence (Price_call, Delta):

Trapezoid dx |        Price |        Delta || Laguerre nodes |        Price |        Delta
--------------------------------------------------------------------------------
         0.2 | 19.4536631436514 | 0.6729664635644601 ||              8 | 19.455037327231672 | 0.6730413740393388
         0.1 | 19.453662951430346 | 0.6729664620584141 ||             16 | 19.453774317885063 | 0.6729670266690413
        0.05 | 19.453662903434516 | 0.6729664616822988 ||             32 | 19.45378151684544 | 0.6729675139028932
        0.01 | 19.45366288808086 | 0.6729664615619753 ||             64 | 19.453781506697773 | 0.6729675138145977
       0.005 | 19.453662887601098 | 0.6729664615582154 ||                |              |             
       0.001 | 19.453662887447575 | 0.6729664615570122 ||                |              |             
      0.0005 | 19.453662887442775 | 0.6729664615569746 ||                |              |    